In [13]:
import os
from itertools import combinations
from IPython.display import HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.metrics import r2_score as r2
from nltk.metrics.agreement import AnnotationTask

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [67]:
def alpha_df(df, good_only=False):
    data = []
    questions = df.columns
    if good_only: questions = list(set(questions)-set({i for i in questions if len(answers[i].dropna().unique()) > 3}))
    for annotator in df.index:
        for question in questions:
            cell = df[question][annotator]
            if cell is not None : data.append((annotator, question, cell))
    try: return AnnotationTask(data=data).alpha()
    except ZeroDivisionError: return 1

In [15]:
def get_trust_coefs(df, good_only=False):
    alphas = dict()
    for i in range(2, len(df.index)+1):
        for combination in combinations(df.index, i):
            alphas[combination] = alpha_df(df.loc[list(combination)], good_only)
    coefs = dict()
    coef = 1
    sorted_keys = [key for key, val in sorted(alphas.items(), key = lambda ele: ele[1])]
    for i, key in enumerate(sorted_keys):
        if alphas[key] > alphas[sorted_keys[i-1]] : coef += 1
        for annotator in key:
            try: coefs[annotator] += coef*alphas[key]
            except KeyError: coefs[annotator] = coef*alphas[key]
    return {annotator : c/max(coefs.values()) for annotator, c in coefs.items()}

In [16]:
def ident_bad_annot(coefs, good_only=False):
    for annotator, coef in coefs.items():
        if coef <= 0.5: yield annotator

In [17]:
def get_profiles(answers, features, classes=True, verbose=False):
    profiles, reports = {}, {}
    for annotator in answers.index:
        if classes:
            dup = answers.loc[annotator].dropna().duplicated(keep=False)
            y = answers.loc[annotator][dup[dup==True].index] 
            x = features.loc[y.index]
            try: X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.1, stratify=y, random_state=38)
            except ValueError:
                try: X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=len(y.unique())/len(y), stratify=y, random_state=38)
                except ZeroDivisionError: continue
            try: profiles[annotator] = SGDClassifier(random_state=9, loss='hinge', l1_ratio=0.89).fit(X_train, Y_train)
            except ValueError: continue
            reports[annotator] = pd.DataFrame.from_dict(classification_report(Y_test, profiles[annotator].predict(X_test), output_dict=True)).T
            if verbose: print(annotator, reports[annotator]["f1-score"]["weighted avg"])
        else:
            y = answers.loc[annotator].dropna()
            x = features.loc[y.index]
            X_train, X_test, Y_train, Y_test = train_test_split(x, y, random_state=38)
            profiles[annotator] = SGDRegressor(penalty="elasticnet", validation_fraction=0.42, random_state=24, power_t=0.55, n_iter_no_change=9, loss='huber', learning_rate='invscaling', l1_ratio=0.76, eta0=7.61, epsilon=0.081, early_stopping=True).fit(X_train, Y_train)
            reports[annotator] = r2(Y_test, profiles[annotator].predict(X_test))
            if verbose: print(annotator, reports[annotator])
    return profiles

In [18]:
def profiles_to_plots(profiles, columns, path):
    for annotator, profile in profiles.items():
        sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
        coefs = profile.coef_
        n_coefs = coefs.shape
        if len(n_coefs) == 1 or n_coefs[0] == 1:
            fig, axes = plt.subplots(1, 1, figsize=(10, 10))
            if len(n_coefs) == 1: sns.barplot(ax=axes, y=columns, x=coefs, palette="hls")
            else: sns.barplot(ax=axes, y=columns, x=coefs[0], palette="hls")
        else:
            fig, axes = plt.subplots(n_coefs[0], 1, figsize=(10, 30))
            for i, coef in enumerate(coefs):
                sns.barplot(ax=axes[i], y=columns, x=coef, palette="hls")
                axes[i].set_title("label " + str(profile.classes_[i]))
        fig.savefig(path+"/"+annotator +".png")

In [19]:
answers = pd.read_csv("answers.csv", names = ["date_time", "email", "ling", "coref"]+list(range(1, 131))+["comments"])
answers.drop(0, axis=0, inplace=True)
answers.drop("date_time", axis=1, inplace=True)
answers = answers.groupby("email").last()
info, answers = answers[["ling", "coref"]], answers[range(1, 131)]

In [24]:
doubles = {23:128, 38:129, 75:130}
checks = {}
for annotator in answers.index:
    check = []
    for k, v in doubles.items():
        first, last = answers[k][annotator], answers[v][annotator]
        check.append("non vérifié") if None in (first, last) else check.append("approuvé") if first == last else check.append("désapprouvé: "+first+" vs "+last)
    checks[annotator] = check
pd.DataFrame(checks).T.style.applymap(lambda x: "background-color: honeydew" if x == "approuvé" else "background-color: white" if x == "non vérifié" else "background-color: lavenderblush")

,0,1,2
Frederique.bordignon@enpc.fr,non vérifié,non vérifié,non vérifié
anais.halftermeyer@univ-orleans.fr,non vérifié,non vérifié,non vérifié
bergler@cse.concordia.ca,non vérifié,non vérifié,non vérifié
emmanuel.schang@univ-orleans.fr,non vérifié,non vérifié,non vérifié
fanny.krimou@hotmail.fr,non vérifié,non vérifié,non vérifié
goudjoasc@aol.com,non vérifié,non vérifié,non vérifié
ilaine.wang@inalco.fr,désapprouvé: 5 vs 6,désapprouvé: 7 vs 6,approuvé
lisa@kolbasa.fyr,non vérifié,non vérifié,non vérifié
loic.grobol@gmail.com,non vérifié,non vérifié,non vérifié
mariya,approuvé,approuvé,approuvé


In [68]:
answers.drop([128, 129, 130], axis=1, inplace=True)
smaller = {"2" : "1", "3" : "2", "4" : "2", "5" : "3", "6" : "3", "7" : "4"}
gravity = {"0":"grave", "1":"grave", "2":"grave", "3":"pas grave", "4":"pas grave", "5":"pas grave", "6":"pas grave", "7":"pas de fautes"}
centrality = {"0":"fondamental", "1":"périphérique", "2":"périphérique", "3":"fondamental", "4":"fondamental", "5":"périphérique", "6":"périphérique", "7":"pas de fautes"}
small, gravity_eval, centrality_eval = [answers.applymap(lambda x : scale[x] if x in scale.keys() else x) for scale in [smaller, gravity, centrality]]
scale_names = ["standard", "réduit","gravité", "centralité"]
alphas_krip = {"échelle standard": [alpha_df(answers), alpha_df(answers, good_only=True)],
              "échelle réduite": [alpha_df(small), alpha_df(small, good_only=True)],
              "échelle de gravité": [alpha_df(gravity_eval), alpha_df(gravity_eval, good_only=True)],
              "échelle de centralité": [alpha_df(centrality_eval), alpha_df(centrality_eval, good_only=True)]}
print("Alphas de Krippendorff:")
pd.DataFrame.from_dict(alphas_krip, orient="index", columns=["toutes les exemples", "sans mauvaises exemples"])

Alphas de Krippendorff:


,toutes les exemples,sans mauvaises exemples
échelle standard,0.116444,0.252319
échelle réduite,0.169048,0.343351
échelle de gravité,0.247672,0.485453
échelle de centralité,0.169100,0.346860


In [69]:
trust_coefs = {'standard_coef':get_trust_coefs(answers),
               'standard_sans_mauvaises_coef':get_trust_coefs(answers, good_only=True),
               'réduit_coef':get_trust_coefs(small),
               'réduit_sans_mauvaises_coef':get_trust_coefs(small, good_only=True),
               'gravité_coef':get_trust_coefs(gravity_eval),
               'gravité_sans_mauvaises_coef':get_trust_coefs(gravity_eval, good_only=True),
               'centralité_coef':get_trust_coefs(centrality_eval),
               'centralité_sans_mauvaises_coef':get_trust_coefs(centrality_eval, good_only=True)}
bad_annot = {scale:list(ident_bad_annot(annot)) for scale, annot in trust_coefs.items()}
if not(any(bad_annot.values())): print("Il n'y a pas d'annotateurs douteaux.")
else:
    print("Annotateurs douteux:")
    display(pd.DataFrame({scale:["❌" if annotator in annot else None for annotator in answers.index] for scale, annot in bad_annot.items()}, index=answers.index).dropna(how="all", axis=0).dropna(how="all", axis=1).fillna(""))

Annotateurs douteux:


,standard_sans_mauvaises_coef,centralité_sans_mauvaises_coef
email,,
ilaine.wang@inalco.fr,,❌
mariya,❌,


In [77]:
alphas_krip = {"échelle standard": [alpha_df(answers[~answers.index.isin(bad_annot["standard_coef"])]), alpha_df(answers[~answers.index.isin(bad_annot["standard_sans_mauvaises_coef"])], good_only=True)],
              "échelle réduite": [alpha_df(small[~answers.index.isin(bad_annot["réduit_coef"])]), alpha_df(small[~answers.index.isin(bad_annot["réduit_sans_mauvaises_coef"])], good_only=True)],
              "échelle de gravité": [alpha_df(gravity_eval[~answers.index.isin(bad_annot["gravité_coef"])]), alpha_df(gravity_eval[~answers.index.isin(bad_annot["gravité_sans_mauvaises_coef"])], good_only=True)],
              "échelle de centralité": [alpha_df(centrality_eval[~answers.index.isin(bad_annot["centralité_coef"])]), alpha_df(centrality_eval[~answers.index.isin(bad_annot["centralité_sans_mauvaises_coef"])], good_only=True)]}
print("Alphas de Krippendorff sans mauvaises annotateurs:")
pd.DataFrame.from_dict(alphas_krip, orient="index", columns=["toutes les exemples", "sans mauvaises exemples"])

Alphas de Krippendorff sans mauvaises annotateurs:


,toutes les exemples,sans mauvaises exemples
échelle standard,0.116444,0.274267
échelle réduite,0.169048,0.343351
échelle de gravité,0.247672,0.485453
échelle de centralité,0.169100,0.423513


In [27]:
info1 = pd.concat([info, pd.DataFrame(trust_coefs)], axis=1, join="inner").round(3)
info1.insert(loc=2, column='average', value=info1.loc[:, ~info1.columns.isin(['ling', 'coref'])].mean(axis=1).round(3))
display(info1.sort_values(by="average", ascending=False).style.set_properties(**{'background-color': 'floralwhite'}, subset=['average']))

,ling,coref,average,standard_coef,standard_sans_mauvaises_coef,réduit_coef,réduit_sans_mauvaises_coef,gravité_coef,gravité_sans_mauvaises_coef,centralité_coef,centralité_sans_mauvaises_coef
sylvie.billot@univ-orleans.fr,non,oui,0.884000,0.984000,0.709000,0.938000,0.839000,1.000000,1.000000,0.887000,0.712000
fanny.krimou@hotmail.fr,oui,non,0.851000,0.829000,0.936000,0.731000,0.934000,0.727000,0.961000,0.721000,0.968000
ygor.gallina@univ-nantes.fr,non,non,0.851000,0.816000,0.936000,0.750000,0.934000,0.702000,0.961000,0.743000,0.968000
Frederique.bordignon@enpc.fr,oui,non,0.835000,0.768000,1.000000,0.667000,1.000000,0.634000,0.990000,0.624000,1.000000
emmanuel.schang@univ-orleans.fr,oui,oui,0.834000,0.799000,0.936000,0.691000,0.934000,0.746000,0.961000,0.633000,0.968000
bergler@cse.concordia.ca,non,oui,0.824000,0.769000,0.936000,0.685000,0.934000,0.656000,0.961000,0.683000,0.968000
lisa@kolbasa.fyr,oui,oui,0.816000,0.754000,0.906000,0.675000,0.912000,0.638000,0.935000,0.727000,0.979000
anais.halftermeyer@univ-orleans.fr,oui,oui,0.813000,0.736000,0.736000,0.782000,0.950000,0.700000,0.997000,0.748000,0.858000
loic.grobol@gmail.com,oui,oui,0.807000,0.984000,0.856000,0.719000,0.810000,0.705000,0.920000,0.749000,0.717000
ilaine.wang@inalco.fr,oui,non,0.783000,1.000000,0.803000,0.833000,0.640000,0.951000,0.903000,0.686000,0.448000


In [28]:
good_index = list(set(answers.columns)-{i for i in answers.columns if len(answers[i].dropna().unique()) > 3})
metrics = pd.read_csv("scores.csv", index_col="id").drop("Unnamed: 0", axis=1).drop("name", axis=1)
recalls = metrics.drop("MUC_P", axis=1).drop("MUC_F1", axis=1).drop("B3_P", axis=1).drop("B3_F1", axis=1).drop("CEAFm_P", axis=1).drop("CEAFm_F1", axis=1).drop("CEAFe_P", axis=1).drop("CEAFe_F1", axis=1).drop("BLANC_P", axis=1).drop("BLANC_F1", axis=1).drop("LEA_P", axis=1).drop("LEA_F1", axis=1).drop("CoNLL_2012", axis=1).T
precisions = metrics.drop("MUC_R", axis=1).drop("MUC_F1", axis=1).drop("B3_R", axis=1).drop("B3_F1", axis=1).drop("CEAFm_R", axis=1).drop("CEAFm_F1", axis=1).drop("CEAFe_R", axis=1).drop("CEAFe_F1", axis=1).drop("BLANC_R", axis=1).drop("BLANC_F1", axis=1).drop("LEA_R", axis=1).drop("LEA_F1", axis=1).drop("CoNLL_2012", axis=1).T
fs = metrics.drop("MUC_R", axis=1).drop("MUC_P", axis=1).drop("B3_R", axis=1).drop("B3_P", axis=1).drop("CEAFm_R", axis=1).drop("CEAFm_P", axis=1).drop("CEAFe_R", axis=1).drop("CEAFe_P", axis=1).drop("BLANC_R", axis=1).drop("BLANC_P", axis=1).drop("LEA_R", axis=1).drop("LEA_P", axis=1).T
a_mean = answers[good_index].astype(float).mean()
a_mode = answers[good_index].astype(float).mode().mean()
s_mean = small[good_index].astype(float).mean()*1.75
s_mode = small[good_index].astype(float).mode().mean()*1.75
gravity_to_num = {"grave":0., "pas grave":1., "pas de fautes":2.}
g_mean = gravity_eval.applymap(lambda x : gravity_to_num[x] if x in gravity_to_num.keys() else x).mean()*3.5
g_mode = gravity_eval.applymap(lambda x : gravity_to_num[x] if x in gravity_to_num.keys() else x).mode().mean()*3.5
bars = {"moyenne":{i:pd.Series([a_mean[i], s_mean[i], g_mean[i], fs[i]["MUC_F1"]*7, fs[i]["B3_F1"]*7, fs[i]["CEAFm_F1"]*7, fs[i]["CEAFe_F1"]*7, fs[i]["BLANC_F1"]*7, fs[i]["LEA_F1"]*7, fs[i]["CoNLL_2012"]*7], index=["standard", "réduit", "gravité_moyenne", "MUC", "B_CUBED", "CEAFm", "CEAFe", "BLANC", "LEA", "CoNLL"]) for i in good_index}, "mode":{i:pd.Series([a_mode[i], s_mode[i], g_mode[i], fs[i]["MUC_F1"]*7, fs[i]["B3_F1"]*7, fs[i]["CEAFm_F1"]*7, fs[i]["CEAFe_F1"]*7, fs[i]["BLANC_F1"]*7, fs[i]["LEA_F1"]*7, fs[i]["CoNLL_2012"]*7], index=["standard", "réduit", "gravité_moyenne", "MUC", "B_CUBED", "CEAFm", "CEAFe", "BLANC", "LEA", "CoNLL"]) for i in good_index}}


head = "<table><tbody>"
for bar in bars:
    row = "<tr><th>{}</th>".format(bar)
    for name, series in bars[bar].items():
        s = series.copy()
        s.name=name
        row += "<td>{}</td>".format(s.to_frame().style.bar(align="mid", color='#5fba7d').render())
    row += '</tr>'
    head += row

head+= "</tbody></table>"

print("Comparaison naive (de 0 à 7, des échelles plus petites sont multipliés par des coefficients pour obtenir 7 comme maximum):")
HTML(head)

Comparaison naive (de 0 à 7, des échelles plus petites sont multipliés par des coefficients pour obtenir 7 comme maximum):


In [34]:
%%capture
errors = pd.read_csv("errors.csv", sep=";", index_col="id", header=0, names = ["id", "frontiers", "type", "bruit", "silence", "non-pertinentes", "absence"])
profiles = {"annotators": get_profiles(answers, errors, verbose=True), "small": get_profiles(small, errors, verbose=True), "gravity": get_profiles(gravity_eval.applymap(lambda x : gravity_to_num[x] if x in gravity_to_num.keys() else x), errors, classes=False, verbose=True), "metrics": get_profiles(fs, errors, classes=False, verbose=True)}
profiles, reports = {}, {}
for name, profile in profiles.items():
    path = "profiles/"+name
    if not(os.path.isdir(path)): os.mkdir(path)
    profiles_to_plots(profile, errors.columns, path)